## Function-Calling (Tool Use) with Converse API in Amazon Bedrock - Advanced scenarios

In this [Function calling tool use with the Bedrock Converse API](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/function-calling/Function_calling_tool_use_with_Converse_API.ipynb) example notebook, we explored basic tool use, in this notebook we will examine use-cases that _require multiple functions to be performed sequentially in the correct order_  as well as use-cases that can benefit from _parallel function calling_. This notebook uses some of the same code that has been written in [Function calling tool use with the Bedrock Converse API](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/function-calling/Function_calling_tool_use_with_Converse_API.ipynb) notebook so it is **_highly recommended that you run that notebook first and familiarize yourself with the basics of function calling_**.

The [Converse or ConverseStream](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html) API is a unified structured text action for simplifying the invocations to Bedrock LLMs. It includes the possibility to define tools for implementing external functions that can be called or triggered from the LLMs.



In [19]:
!pip3 install -qU boto3

In [20]:
import json
import time
import boto3
import asyncio
import logging
import inspect
from typing import List
from pydantic import Field, create_model

In [21]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [22]:
# we will be using the Claude 3 Sonnet for this notebook
MODEL_ID: str = "anthropic.claude-3-sonnet-20240229-v1:0"

In [23]:
bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = boto3.session.Session().region_name,
    )

In [24]:
# decorator to generate tool config for any function
def bedrock_tool(name, description):
    def decorator(func):
        input_model = create_model(
            func.__name__ + "_input",
            **{
                name: (param.annotation, param.default)
                for name, param in inspect.signature(func).parameters.items()
                if param.default is not inspect.Parameter.empty
            },
        )

        func.bedrock_schema = {
            'toolSpec': {
                'name': name,
                'description': description,
                'inputSchema': {
                    'json': input_model.schema()
                }
            }
        }
        return func

    return decorator

### Tool definition
In this notebook we have some simple functions that respond with weather related information. The choice of these functions is such that for some use-cases it would be required to use multiple functions in a given order and it is this orchestration that we are relying on the LLM to perform correctly.

In [25]:
### DEFINE YOUR TOOLS HERE ###
class ToolsList:
    @bedrock_tool(
        name="get_weather",
        description="Get weather of a location."
    )
    def get_weather(self, city: str = Field(..., description="City of the location"),
                    state: str = Field(..., description="State of the location")):
        result = f'Weather in {city, state} is 70F and clear skies.'
        return result
    
    @bedrock_tool(
        name="get_my_fav_city",
        description="Get name of my favorite city."
    )
    def get_my_fav_city(self):
        fav_city = 'Georgetown, D.C.'
        return fav_city
    
    @bedrock_tool(
        name="get_my_fav_month",
        description="Get name of my favorite month."
    )
    def get_my_fav_month(self):
        fav_month = 'February'
        return fav_month
    
    @bedrock_tool(
        name="get_weather_in_month",
        description="Get weather of a location in a given month."
    )
    def get_weather_in_month(self, city: str = Field(..., description="City of the location"),
                    state: str = Field(..., description="State of the location"),
                    month: str = Field(..., description="Month of interest")):
        result = f'Weather in {city, state, month} is 70F and clear skies.'
        return result

In [26]:
toolConfig = {
    'tools': [tool.bedrock_schema for tool in ToolsList.__dict__.values() if hasattr(tool, 'bedrock_schema')],
    'toolChoice': {'auto': {}}
}
toolConfig

{'tools': [{'toolSpec': {'name': 'get_weather',
    'description': 'Get weather of a location.',
    'inputSchema': {'json': {'properties': {'city': {'description': 'City of the location',
        'title': 'City',
        'type': 'string'},
       'state': {'description': 'State of the location',
        'title': 'State',
        'type': 'string'}},
      'required': ['city', 'state'],
      'title': 'get_weather_input',
      'type': 'object'}}}},
  {'toolSpec': {'name': 'get_my_fav_city',
    'description': 'Get name of my favorite city.',
    'inputSchema': {'json': {'properties': {},
      'title': 'get_my_fav_city_input',
      'type': 'object'}}}},
  {'toolSpec': {'name': 'get_my_fav_month',
    'description': 'Get name of my favorite month.',
    'inputSchema': {'json': {'properties': {},
      'title': 'get_my_fav_month_input',
      'type': 'object'}}}},
  {'toolSpec': {'name': 'get_weather_in_month',
    'description': 'Get weather of a location in a given month.',
    'input

### Setup Python `asyncio` based concurrency so that we can call multiple functions in parallel

For some use-cases the LLM correctly identifies that we need to make multiple calls and these calls can be made in parallel to save on the overall latency. The execution of the functions is actually done by our code so we are in control if we want to invoke these functions sequentially or in parallel.

In [27]:

def call_function(function, tool_class):
    logger.info(f"function={function}")
    function = function[1]
    logger.info(f"step 2, function - Calling tool...{function['name']}")
    tool_name = function['name']
    tool_args = function['input'] or {}
    tool_response = getattr(tool_class, tool_name)(**tool_args)
    logger.info(f"step 2, function {function['name']}- Got tool response...{tool_response}")
    return dict(tool_response=tool_response, tool_use_id=function['toolUseId'])

# Asynchronous wrapper function to allow our function calls to happen concurrently
async def async_call_function(function, tool_class):
    # Run the call_function function in a separate thread to run each function asynchronously
    return await asyncio.to_thread(call_function, function, tool_class)

# Final asynchronous function to deploy all of the functions concurrently
async def async_call_all_functions(function_calling, tool_class):
    
    n: int = 4 # max concurrency so as to not get a throttling exception
    
    ## Split experiments into smaller batches for concurrent deployment
    function_calling_splitted = [function_calling[i * n:(i + 1) * n] for i in range((len(function_calling) + n - 1) // n )]
    results = []
    for function_sublist in function_calling_splitted:
        ## do the function calls in batches
        result = await asyncio.gather(*[async_call_function(function, 
                                                            tool_class) for function in enumerate(function_sublist)])
        ## Collect and furthermore extend the results from each batch
        results.extend(result)
    return results

### Wrapper functions for the Bedrock `Converse` API

In [28]:
def converse_with_tools(modelId, messages, system='', toolConfig=None):
    return bedrock.converse(
        modelId=modelId,
        system=system,
        messages=messages,
        toolConfig=toolConfig
    )

### Anatomy of a function call conversation
This function uses function call to answer uses questions in 3 steps
1. Ask the LLM what to do given the user question and the tools provided.
1. Read the LLM response and if the response says use this tool(s) then invoke the
   function(s) corresponding to that tool.
   1. If the LLM says no tool use is required/available then return that as the final 
   response
1. Provide the entire conversation history including the function call output to the LLM
   and ask for a response.
1. Repeat steps 2 and 3 until the LLM response says no more tools need to be used, return
   that as the final response.   

In [29]:
async def converse(tool_class, modelId, prompt, system='', toolConfig=None):

    """
    This function uses function call to answer uses questions in 3 steps
    1. Ask the LLM what to do given the user question and the tools provided.
    2. Read the LLM response and if the response says use this tool(s) then invoke the
       function(s) corresponding to that tool.
       - If the LLM says no tool use is required/available then return that as the final 
         response
    3. Provide the entire conversation history including the function call output to the LLM
       and ask for a response.
    4. Repeat steps 2 and 3 until the LLM response says no more tools need to be used, return
       that as the final response.   
    """
    # step 1. invoke model for the first time to figure out what tools if any are needed
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    logger.info(f"step 1. Invoking model...{modelId}")
    output = converse_with_tools(modelId, messages, system, toolConfig)
    messages.append(output['output']['message'])
    logger.info(f"step 1 output from model...{json.dumps(output['output'], indent=2, default=str)}")

    while True:
        # step 2. check if the model said any tools should be used, invoke the tools that the model
        # said need to be used

        function_calling = [c['toolUse'] for c in output['output']['message']['content'] if 'toolUse' in c]
        if function_calling:
            tool_result_message = {"role": "user", "content": []}
            logger.info(f"there are {len(function_calling)} entries in function_calling")
            # async version
            s = time.perf_counter()

            # Call all functions in parallel
            tool_responses_list = await async_call_all_functions(function_calling, tool_class)
            elapsed_async = time.perf_counter() - s
            logger.info(f"ran {len(function_calling)} in parallel in {elapsed_async:0.4f} seconds")
            for r in tool_responses_list:
                tool_result_message['content'].append({
                    'toolResult': {
                        'toolUseId': r['tool_use_id'],
                        'content': [{"text": r['tool_response']}]
                    }
                })
            messages.append(tool_result_message)
        else:
            logger.info(f"there are NO functions to call as per the LLM")
            break

        # step 3. call the model one final time to put all the tool responses together
        # and generate a final response
        logger.info(f"step 3, calling model with the results from calling {len(function_calling)} functions")
        
        output = converse_with_tools(modelId, messages, system, toolConfig)
        messages.append(output['output']['message'])
        logger.info(f"function calling - Got final answer from step 3, checking if more function calling is needed")
        function_calling = [c['toolUse'] for c in output['output']['message']['content'] if 'toolUse' in c]
        if len(function_calling) == 0:
            logger.info(f"step 3. no more function calling is needed, we have our final answer, exiting")
            break
        else:
            logger.info(f"step 3. seems like we still need to call {len(function_calling)} functions, continuing..")


    return messages, output

#### Set the system prompt
The system prompt is set for the overall task that this "agent" is expected to perform. Do not make this very specific to one particular use-case.

In [30]:
### ADJUST YOUR SYSTEM PROMPT HERE - IF DESIRED ###
system_prompt: List[str] = [{"text": "You're provided with multiple tools that can help you answer user questions about weather; \
                              only use a tool if required. You can call the tool multiple times in the same response if required. \
                              Given a user input first think about which all tools would be needed and then include them in your function calling response in \
                              the correct order.\
                              Don't make reference to the tools in your final answer."}]

### Use-case 1. No tool use
Ask an LLM a question that it cannot answer based on the tools provided.

In [31]:
### REPLACE WITH YOUR OWN PROMPTS HERE ###
prompt: str = "What is the #1 song in Paris?"

messages, output = await converse(ToolsList(), MODEL_ID, prompt, system_prompt, toolConfig)
logger.info(f"final response = {json.dumps(output['output'], indent=2, default=str)}")
logger.info(f"Output:\n{output['output']['message']['content'][0].get('text')}\n")
#logger.info(f"Messages:\n{json.dumps(messages, indent=2, ensure_ascii=False)}\n")

[2024-06-26 22:36:17,805] p43052 {1303253139.py:17} INFO - step 1. Invoking model...anthropic.claude-3-sonnet-20240229-v1:0
[2024-06-26 22:36:21,378] p43052 {1303253139.py:20} INFO - step 1 output from model...{
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "Unfortunately, I don't have any tools that can provide information about the top songs or music charts in a specific city like Paris. My tools are focused on providing weather information for different locations. I don't have the capability to look up music chart data."
      }
    ]
  }
}
[2024-06-26 22:36:21,379] p43052 {1303253139.py:46} INFO - there are NO functions to call as per the LLM
[2024-06-26 22:36:21,380] p43052 {2903005534.py:5} INFO - final response = {
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "Unfortunately, I don't have any tools that can provide information about the top songs or music charts in a specific city like Paris. My tools are focu

### Use-case 2: simple use-case call one function
Ask the LLM to find the weather in Paris and it says ok call the get weather tool/function, we call the get weather function and provide the tool response and original question to the LLM and ask it again and this time it provides the final answer.

In [32]:
prompt = "what is the weather in Paris?"
# prompt = "what is the weather in my favorite city in my favorite month?"

messages, output = await converse(ToolsList(), MODEL_ID, prompt, system_prompt, toolConfig)
logger.info(f"final response = {json.dumps(output['output'], indent=2, default=str)}")
logger.info(f"Output:\n{output['output']['message']['content'][0].get('text')}\n")
#logger.info(f"Messages:\n{json.dumps(messages, indent=2, ensure_ascii=False)}\n")

[2024-06-26 22:36:21,393] p43052 {1303253139.py:17} INFO - step 1. Invoking model...anthropic.claude-3-sonnet-20240229-v1:0
[2024-06-26 22:36:23,393] p43052 {1303253139.py:20} INFO - step 1 output from model...{
  "message": {
    "role": "assistant",
    "content": [
      {
        "toolUse": {
          "toolUseId": "tooluse_mXDD6tDvR9SpnxcGqaR-xg",
          "name": "get_weather",
          "input": {
            "city": "Paris",
            "state": "NA"
          }
        }
      }
    ]
  }
}
[2024-06-26 22:36:23,395] p43052 {1303253139.py:29} INFO - there are 1 entries in function_calling
[2024-06-26 22:36:23,396] p43052 {3670242087.py:2} INFO - function=(0, {'toolUseId': 'tooluse_mXDD6tDvR9SpnxcGqaR-xg', 'name': 'get_weather', 'input': {'city': 'Paris', 'state': 'NA'}})
[2024-06-26 22:36:23,397] p43052 {3670242087.py:4} INFO - step 2, function - Calling tool...get_weather
[2024-06-26 22:36:23,398] p43052 {3670242087.py:8} INFO - step 2, function get_weather- Got tool response

ran 1 in parallel in 0.00 seconds


[2024-06-26 22:36:24,931] p43052 {1303253139.py:55} INFO - function calling - Got final answer from step 3, checking if more function calling is needed
[2024-06-26 22:36:24,932] p43052 {1303253139.py:58} INFO - step 3. no more function calling is needed, we have our final answer, exiting
[2024-06-26 22:36:24,933] p43052 {4188552546.py:5} INFO - final response = {
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "The weather in Paris is 70\u00b0F and clear skies."
      }
    ]
  }
}
[2024-06-26 22:36:24,934] p43052 {4188552546.py:6} INFO - Output:
The weather in Paris is 70°F and clear skies.



#### Use case 3 - Reason through multiple functions in a sequential order
We are for weather in "my favorite" city in "my favorite month" so the LLM suggests call the my favorite city function first, then we call that function and provide the output to the LLM again adn this time it says find my favorite month, we provide the combined output available at this point to the LLM and this time it says ok call the get weather API and then that is the final answer.

In [33]:
prompt = "what is the weather in my favorite city in my favorite month?"

messages, output = await converse(ToolsList(), MODEL_ID, prompt, system_prompt, toolConfig)
logger.info(f"final response = {json.dumps(output['output'], indent=2, default=str)}")
logger.info(f"Output:\n{output['output']['message']['content'][0].get('text')}\n")
#logger.info(f"Messages:\n{json.dumps(messages, indent=2, ensure_ascii=False)}\n")

[2024-06-26 22:36:24,948] p43052 {1303253139.py:17} INFO - step 1. Invoking model...anthropic.claude-3-sonnet-20240229-v1:0
[2024-06-26 22:36:26,826] p43052 {1303253139.py:20} INFO - step 1 output from model...{
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "Okay, let me try to gather the required information to answer your question:"
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_jzNj8wtFSmWyWcI73es0Qw",
          "name": "get_my_fav_city",
          "input": {}
        }
      }
    ]
  }
}
[2024-06-26 22:36:26,828] p43052 {1303253139.py:29} INFO - there are 1 entries in function_calling
[2024-06-26 22:36:26,829] p43052 {3670242087.py:2} INFO - function=(0, {'toolUseId': 'tooluse_jzNj8wtFSmWyWcI73es0Qw', 'name': 'get_my_fav_city', 'input': {}})
[2024-06-26 22:36:26,830] p43052 {3670242087.py:4} INFO - step 2, function - Calling tool...get_my_fav_city
[2024-06-26 22:36:26,831] p43052 {3670242087.py:8} INFO - step 2, function ge

ran 1 in parallel in 0.00 seconds


[2024-06-26 22:36:28,429] p43052 {1303253139.py:55} INFO - function calling - Got final answer from step 3, checking if more function calling is needed
[2024-06-26 22:36:28,430] p43052 {1303253139.py:61} INFO - step 3. seems like we still need to call 1 functions, continuing..
[2024-06-26 22:36:28,431] p43052 {1303253139.py:29} INFO - there are 1 entries in function_calling
[2024-06-26 22:36:28,432] p43052 {3670242087.py:2} INFO - function=(0, {'toolUseId': 'tooluse_WwppVe2iSB2jnyADaDcBZQ', 'name': 'get_my_fav_month', 'input': {}})
[2024-06-26 22:36:28,432] p43052 {3670242087.py:4} INFO - step 2, function - Calling tool...get_my_fav_month
[2024-06-26 22:36:28,433] p43052 {3670242087.py:8} INFO - step 2, function get_my_fav_month- Got tool response...February
[2024-06-26 22:36:28,434] p43052 {1303253139.py:51} INFO - step 3, calling model with the results from calling 1 functions


ran 1 in parallel in 0.00 seconds


[2024-06-26 22:36:31,135] p43052 {1303253139.py:55} INFO - function calling - Got final answer from step 3, checking if more function calling is needed
[2024-06-26 22:36:31,136] p43052 {1303253139.py:61} INFO - step 3. seems like we still need to call 1 functions, continuing..
[2024-06-26 22:36:31,137] p43052 {1303253139.py:29} INFO - there are 1 entries in function_calling
[2024-06-26 22:36:31,140] p43052 {3670242087.py:2} INFO - function=(0, {'toolUseId': 'tooluse_85jJPG5kQMiCWTvsN1Y4tA', 'name': 'get_weather_in_month', 'input': {'city': 'Georgetown', 'state': 'D.C.', 'month': 'February'}})
[2024-06-26 22:36:31,142] p43052 {3670242087.py:4} INFO - step 2, function - Calling tool...get_weather_in_month
[2024-06-26 22:36:31,143] p43052 {3670242087.py:8} INFO - step 2, function get_weather_in_month- Got tool response...Weather in ('Georgetown', 'D.C.', 'February') is 70F and clear skies.
[2024-06-26 22:36:31,145] p43052 {1303253139.py:51} INFO - step 3, calling model with the results fr

ran 1 in parallel in 0.01 seconds


[2024-06-26 22:36:32,602] p43052 {1303253139.py:55} INFO - function calling - Got final answer from step 3, checking if more function calling is needed
[2024-06-26 22:36:32,602] p43052 {1303253139.py:58} INFO - step 3. no more function calling is needed, we have our final answer, exiting
[2024-06-26 22:36:32,603] p43052 {309264306.py:4} INFO - final response = {
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "The weather in your favorite city Georgetown, D.C. in your favorite month February is 70F and clear skies."
      }
    ]
  }
}
[2024-06-26 22:36:32,604] p43052 {309264306.py:5} INFO - Output:
The weather in your favorite city Georgetown, D.C. in your favorite month February is 70F and clear skies.



#### Use-case 4 - Parallel function calling

We ask for weather in two cities, the LLM results 2 function calls and then we run these functions calls in parallel.

In [34]:
prompt = "What is the weather in Paris and in Berlin?"

messages, output = await converse(ToolsList(), MODEL_ID, prompt, system_prompt, toolConfig)
logger.info(f"final response = {json.dumps(output['output'], indent=2, default=str)}")
logger.info(f"Output:\n{output['output']['message']['content'][0].get('text')}\n")

[2024-06-26 22:36:32,618] p43052 {1303253139.py:17} INFO - step 1. Invoking model...anthropic.claude-3-sonnet-20240229-v1:0
[2024-06-26 22:36:36,196] p43052 {1303253139.py:20} INFO - step 1 output from model...{
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "Okay, let me get the weather information for Paris and Berlin:"
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_iQNDrl3gR7SsuKzSSmNj4Q",
          "name": "get_weather",
          "input": {
            "city": "Paris",
            "state": "France"
          }
        }
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_pOncmu_ETy-g01TQrPlVqw",
          "name": "get_weather",
          "input": {
            "city": "Berlin",
            "state": "Germany"
          }
        }
      }
    ]
  }
}
[2024-06-26 22:36:36,197] p43052 {1303253139.py:29} INFO - there are 2 entries in function_calling
[2024-06-26 22:36:36,198] p43052 {3670242087.py:2} INFO - fun

ran 2 in parallel in 0.01 seconds


[2024-06-26 22:36:39,957] p43052 {1303253139.py:55} INFO - function calling - Got final answer from step 3, checking if more function calling is needed
[2024-06-26 22:36:39,958] p43052 {1303253139.py:58} INFO - step 3. no more function calling is needed, we have our final answer, exiting
[2024-06-26 22:36:39,959] p43052 {3499383132.py:4} INFO - final response = {
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "The weather in Paris, France is 70F and clear skies.\nThe weather in Berlin, Germany is also 70F and clear skies."
      }
    ]
  }
}
[2024-06-26 22:36:39,960] p43052 {3499383132.py:5} INFO - Output:
The weather in Paris, France is 70F and clear skies.
The weather in Berlin, Germany is also 70F and clear skies.

